In [10]:
import pandas as pd
import gcsfs

## Load first 600K rows of Items Metadata

In [2]:
items_metadata_file_path = 'gs://amazon-home-and-kitchen/meta_Home_and_Kitchen.jsonl'

# Initialize GCS filesystem
fs = gcsfs.GCSFileSystem()

# Open the file from the GCS bucket
with fs.open(items_metadata_file_path, 'r') as f:
    # Read the JSONL file in chunks to handle large files efficiently
    #json_reader = pd.read_json(f, lines=True, chunksize=500000)
    
    # Read the JSONL file in 600K rows at a time
    json_reader = pd.read_json(f, lines=True, chunksize=600000)
    
    # for i, chunk in enumerate(json_reader):
    #     if i == 0:
    #         meta_df = chunk
    #     else:
    #         meta_chunk = chunk
    #         meta_df = pd.concat([meta_df, meta_chunk])
    
    # for i, chunk in enumerate(json_reader):
    #     if i == 0:
    #         meta_df = chunk
    #     elif i > 0 and i <= 3:
    #         meta_chunk = chunk
    #         meta_df = pd.concat([meta_df, meta_chunk])
    #     elif i > 3:
    #         break
    
    ## Optionally, you can concatenate all chunks into a single DataFrame
    #full_meta_df = pd.concat(json_reader, ignore_index=True)
        
    for i, chunk in enumerate(json_reader):
        if i == 0:  # You can choose which chunk to process, e.g., the first chunk
            meta_df = chunk  # This is the DataFrame for the first chunk
            break

### Check the number of rows in the Items Metadata DataFrame.

In [3]:
len(meta_df)

600000

### Display the names of the columns in the Items Metadata DataFrame.

In [4]:
meta_df.columns

Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'bought_together', 'subtitle', 'author'],
      dtype='object')

In [7]:
meta_df['main_category'].unique()

array(['Amazon Home', None, 'Tools & Home Improvement', 'Handmade',
       'AMAZON FASHION', 'Office Products', 'Industrial & Scientific',
       'Pet Supplies', 'Sports & Outdoors', 'Health & Personal Care',
       'All Electronics', 'All Beauty', 'Baby', 'Toys & Games',
       'Portable Audio & Accessories', 'Home Audio & Theater',
       'Appliances', 'Arts, Crafts & Sewing', 'Musical Instruments',
       'Cell Phones & Accessories', 'Grocery', 'Automotive', 'Computers',
       'Camera & Photo', 'Video Games', 'Premium Beauty', 'Software',
       'Car Electronics', 'Entertainment', 'Amazon Devices',
       'Collectible Coins', 'Audible Audiobooks', 'Books', 'Gift Cards',
       'Sports Collectibles'], dtype=object)

### Exclude unneeded columns in the Items Metadata DataFrame and preview the data.

In [4]:
meta_df = meta_df[["parent_asin", "main_category", "title", "average_rating", "price"]]
meta_df.head()

,parent_asin,main_category,title,average_rating,price
0,B07R3DYMH6,Amazon Home,Set of 4 Irish Coffee Glass Mugs Footed 10.5 o...,4.6,24.95
1,B0BNZ8Q7YT,Amazon Home,Foaming Soap Dispenser Thick Ceramic Foam Hand...,4.4,24.99
2,B01508WQC6,Amazon Home,Tapestry Trading 558W90 90 in. European Lace T...,5.0,45.64
3,B00KKU8HTG,Amazon Home,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,None
4,B0B61RJ848,Amazon Home,Chisander 20 Inches Grey with White Super Soft...,4.6,9.99


## Load User Reviews Train Data

In [3]:
# Set up your GCS file path
user_reviews_train_file_path = 'gs://amazon-home-and-kitchen/Home_and_Kitchen_Train.jsonl'

# Open the file from the GCS bucket
with fs.open(user_reviews_train_file_path, 'r') as f:
    # Read the JSONL file in chunks to handle large files efficiently
    #json_reader = pd.read_json(f, lines=True, chunksize=1000000)
    
    # Read the JSONL file in 500K rows at a time
    json_reader = pd.read_json(f, lines=True, chunksize=500000)
    
    ## Optionally, you can concatenate all chunks into a single DataFrame
    #full_reviews_df = pd.concat(json_reader, ignore_index=True)
    
    for i, chunk in enumerate(json_reader):
        if i == 0:
            reviews_df = chunk
        else:
            reviews_chunk = chunk
            reviews_df = pd.concat([reviews_df, reviews_chunk], ignore_index=True)

    # for i, chunk in enumerate(json_reader):
    #     if i == 0:  # You can choose which chunk to process, e.g., the first chunk
    #         reviews_df = chunk  # This is the DataFrame for the first chunk
    #         break

# Display the first few rows of the DataFrame
#print(reviews_df.head())
#print(reviews_df.columns)

### Check the number of rows in the User Reviews DataFrame.

In [4]:
len(reviews_df)

6740994

### Display the names of the columns in the User Reviews DataFrame.

In [7]:
reviews_df.columns

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

### Exclude unneeded columns in the User Reviews DataFrame and preview the data.

In [8]:
reviews_df = reviews_df[["parent_asin", "verified_purchase", "helpful_vote", "user_id", "text"]]
reviews_df.head()

,parent_asin,verified_purchase,helpful_vote,user_id,text
0,B09XWYG6X1,True,1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,Livid. Once again received an obviously used ...
1,B0BXDLF8TW,True,0,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,I purchased these for multiple reasons. The ma...
2,B09G2PW8ZG,True,0,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,[[VIDEOID:c87e962bc893a948856b0f1b285ce6cc]] I...
3,B08CSZDXZY,True,0,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,If you live at a higher elevation like me (5k ...
4,B0C6V27S6N,True,1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,I use these to store yarn. They easily hold 12...


### Join Items Metadata DataFrame to User Reviews Train DataFrame.

In [5]:
reviews_with_meta_df = pd.merge(meta_df, reviews_df, left_on='parent_asin', right_on='parent_asin')

### How many reviews (rows) are there in the resulting DataFrame?

In [11]:
from google.cloud import storage


# Specify GCS bucket and path
bucket_name = 'amazon-home-and-kitchen'
destination_blob_name = 'full_train_data.csv'

# Save DataFrame as CSV locally first
reviews_with_meta_df.to_csv('/tmp/full_train.csv', index=False)

# Initialize a GCS client and upload wwthe file
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename('/tmp/full_train.csv')

In [12]:
reviews_with_meta_df.columns

Index(['main_category', 'title_x', 'average_rating', 'rating_number',
       'features', 'description', 'price', 'images_x', 'videos', 'store',
       'categories', 'details', 'parent_asin', 'bought_together', 'subtitle',
       'author', 'rating', 'title_y', 'text', 'images_y', 'asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

In [ ]:
len(reviews_with_meta_df)

In [10]:
for category in reviews_with_meta_df['main_category'].unique():
    if category != None:
        print('# rows in ' + category + ': ' + str(len(meta_df[meta_df['main_category'] == category])))

# rows in Amazon Home: 510915
# rows in Tools & Home Improvement: 28429
# rows in AMAZON FASHION: 4920
# rows in Office Products: 4087
# rows in Pet Supplies: 420
# rows in Sports & Outdoors: 2705
# rows in Health & Personal Care: 5138
# rows in Baby: 1578
# rows in Toys & Games: 7371
# rows in Handmade: 1940
# rows in Home Audio & Theater: 527
# rows in Appliances: 2326
# rows in Arts, Crafts & Sewing: 2430
# rows in Cell Phones & Accessories: 666
# rows in Industrial & Scientific: 5901
# rows in Automotive: 738
# rows in Grocery: 599
# rows in All Beauty: 1391
# rows in Computers: 222
# rows in All Electronics: 1251
# rows in Camera & Photo: 206
# rows in Musical Instruments: 385
# rows in Video Games: 43
# rows in Car Electronics: 15
# rows in Portable Audio & Accessories: 12
# rows in Entertainment: 18
# rows in Premium Beauty: 15
# rows in Software: 19
# rows in Amazon Devices: 2
# rows in Collectible Coins: 4
# rows in Books: 1
# rows in Gift Cards: 1


### How many unique products are there?

In [13]:
reviews_with_meta_df["parent_asin"].nunique()

269248

### How many unique reviewers are there?

In [14]:
reviews_with_meta_df["user_id"].nunique()

697405

## Preview the resulting data.

In [15]:
reviews_with_meta_df.head(200)

,parent_asin,title,average_rating,price,verified_purchase,helpful_vote,user_id,text
0,B00KKU8HTG,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,None,True,0,AHED326L5JZMCI2GAXFXNKZGQKNA,nice
1,B00KKU8HTG,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,None,True,0,AHROHCEEEGFMNYMMWXQCHLPBVVLQ,These chairs are beautiful bought 4 of these i...
2,B00KKU8HTG,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,None,True,0,AHH4QDZVVTYKMCM7UYL7KD24GIPA,Horible build quality. The stools don't lock i...
3,B00KKU8HTG,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,None,True,0,AHBDGI6TOGE33U5W6VEP55NEKC6Q,"Great quality, especially for the price!<br />..."
4,B00KKU8HTG,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,None,True,0,AHTYSHSOLY4YTIPY53BRRJ2T325Q,Great for the price! Worked perfectly for our ...
...,...,...,...,...,...,...,...,...
195,B0C4CPL2RV,"Walensee Large Bathroom Rug (24 x 60, Black) E...",4.3,41.99,True,0,AGYXPCIVB5UESMNZZUZY7LIY46KA,"So thick, Beautiful Bright Yellow which I love..."
196,B0C4CPL2RV,"Walensee Large Bathroom Rug (24 x 60, Black) E...",4.3,41.99,True,0,AGYXPCIVB5UESMNZZUZY7LIY46KA,So soft and such a bright yellow which is what...
197,B0C4CPL2RV,"Walensee Large Bathroom Rug (24 x 60, Black) E...",4.3,41.99,True,0,AGTLU344CUPJDUN6GUNLGLY7M6DA,Lovely shag rug with vibrant color. Looks grea...
198,B0C4CPL2RV,"Walensee Large Bathroom Rug (24 x 60, Black) E...",4.3,41.99,True,3,AHBEGJ5QFXMEC3AYSS2A3OVE7W5Q,This is a review of Walensee Bathroom Contour ...
